In [341]:
import numpy as np
import pandas as pd
import plotly.express as px

from dask import dataframe as dd
import time
import os
from dask.distributed import Client

fname_311 = '311_Service_Requests_from_2010_to_Present.csv'
fname_zips = 'brooklyn_zip_neighborhood.csv'
fname_population = 'NY_population_by_zip.csv'
fname_rents = 'rent_cleaned.csv'

In [215]:
# rows_to_keep = [::100, :]
print(os.path.getsize(fname_311)/1024/1024/1024 , "GB")


6.567031293176115 GB


In [52]:
client = Client(processes=False)
client

/Users/epavlov/.pyenv/versions/3.7.4/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 58129 instead
  f"Port {expected} is already in use.\n"


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.1.179:58129/status,
Dashboard: http://192.168.1.179:58129/status,Workers: 1
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: False
Comm: inproc://192.168.1.179/27467/21,Workers: 1
Dashboard: http://192.168.1.179:58129/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: inproc://192.168.1.179/27467/24,Total threads: 8
Dashboard: http://192.168.1.179:58130/status,Memory: 16.00 GiB
Nanny: None,


In [49]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [168]:
# could not read 7GB file using pandas, found package 'dask' which provides multithreaded capabilities
start = time.time()
# cols_to_read = [0,1,2,3,5,8,16,19,20,23,24,25]
cols_to_read = [0,1,5,8,16,25]
# dates = ['Created Date'] 
# types = {'Unique Key':object, 'Incident Zip':object}
types = {'Unique Key':object, 'Incident Zip':object}
dask_df = dd.read_csv(fname_311, usecols=cols_to_read, dtype = types)
end = time.time()
print("Read csv with dask: ",(end-start),"sec")
dask_df = dask_df.compute(scheduler='threads')
end1 = time.time()
print("Computed csv with dask: ",(end1-end),"sec")

Read csv with dask:  0.04510998725891113 sec
Computed csv with dask:  47.33354711532593 sec


In [342]:
rent_df = pd.read_csv(fname_rents)

In [343]:
rent_df['Borough'].unique()

array(['Brooklyn'], dtype=object)

In [344]:
rent_df = rent_df[rent_df['Borough']=='Brooklyn']
rent_df['Neighborhood'].unique()

array(['Bath Beach', 'Bay Ridge', 'Bedford-Stuyvesant',
       'Parkville/Bensonhurst', 'Bergen Beach', 'Boerum Hill',
       'Borough Park', 'Brighton Beach', 'Brooklyn', 'Brooklyn Heights',
       'Brownsville', 'Bushwick', 'Canarsie', 'Carroll Gardens',
       'Clinton Hill', 'Cobble Hill', 'Columbia St Waterfront District',
       'Coney Island', 'DUMBO', 'Ditmas Park', 'Downtown Brooklyn',
       'Dyker Heights', 'East Brooklyn', 'East Flatbush', 'East New York',
       'Vanderveer/Flatbush', 'Flatlands', 'Fort Greene', 'Gowanus',
       'Gravesend/Homecrest', 'Greenpoint', 'Greenwood', 'Kensington',
       'Manhattan Beach', 'Marine Park', 'Midwood', 'Mill Basin',
       'North Brooklyn', 'Northwest Brooklyn', 'Park Slope',
       'Prospect Heights', 'Prospect Lefferts Gardens', 'Prospect Park',
       'Prospect Park South', 'Red Hook', 'Sheepshead Bay',
       'South Brooklyn', 'Industry City/Sunset Park', 'Williamsburg',
       'Windsor Terrace', 'Bath Beach/Bensonhurst',
     

In [216]:
# load a file mapping nyc Brooklyn zip codes to neighborhood names
zip_df = pd.read_csv(fname_zips)

In [222]:
zip_df['Zip'].sort_values().unique()


array([11201, 11203, 11204, 11205, 11206, 11207, 11208, 11209, 11210,
       11211, 11212, 11213, 11214, 11215, 11216, 11217, 11218, 11219,
       11220, 11221, 11222, 11223, 11224, 11225, 11226, 11228, 11229,
       11230, 11231, 11232, 11233, 11234, 11235, 11236, 11237, 11238,
       11239, 11249])

In [283]:
pop_df = pd.read_csv(fname_population,thousands=',')
# pop_df['Zip'] = pop_df['Zip'].astype(np.int64)
# pop_df['Population'] = pop_df['Population'].astype(np.int64)
# list(pop_df['Zip'].unique())
pop_df.dtypes

Zip           int64
Population    int64
dtype: object

In [169]:
dask_df.dtypes


Unique Key        object
Created Date      object
Complaint Type    object
Incident Zip      object
City              object
Borough           object
dtype: object

In [170]:
dask_df.head()

,Unique Key,Created Date,Complaint Type,Incident Zip,City,Borough
0,25804767,06/24/2013 11:47:39 AM,Illegal Parking,10306,STATEN ISLAND,STATEN ISLAND
1,25804768,06/24/2013 07:06:14 AM,Illegal Parking,10065,NEW YORK,MANHATTAN
2,25804769,06/24/2013 10:16:53 AM,Illegal Parking,11236,BROOKLYN,BROOKLYN
3,25804770,06/24/2013 04:31:58 PM,Traffic,11367,FLUSHING,QUEENS
4,25804774,06/24/2013 09:26:03 AM,Overgrown Tree/Branches,11214,BROOKLYN,BROOKLYN


In [171]:
# dask_df['Complaint Type'].unique()

In [172]:
dask_df.to_csv("311_short.csv")

In [173]:
dask_df['Created Date'] = pd.to_datetime(dask_df['Created Date'])

In [393]:
# df1 = dask_df.groupby(['Complaint Type']).filter(lambda x : len(x)>5000)

res = dask_df[~(dask_df['Created Date'] < '2013-06-01')]

,Unique Key,Created Date,Complaint Type,Incident Zip,City,Borough
0,25804767,2013-06-24 11:47:39,Illegal Parking,10306,STATEN ISLAND,STATEN ISLAND
1,25804768,2013-06-24 07:06:14,Illegal Parking,10065,NEW YORK,MANHATTAN
2,25804769,2013-06-24 10:16:53,Illegal Parking,11236,BROOKLYN,BROOKLYN
3,25804770,2013-06-24 16:31:58,Traffic,11367,FLUSHING,QUEENS
4,25804774,2013-06-24 09:26:03,Overgrown Tree/Branches,11214,BROOKLYN,BROOKLYN
...,...,...,...,...,...,...
20010,42609622,2019-05-07 22:17:39,Derelict Vehicle,10010,NEW YORK,MANHATTAN
20011,42609623,2019-05-07 18:51:03,Derelict Vehicle,11417,OZONE PARK,QUEENS
20012,"""message"" : ""Internal error""",NaT,NaN,NaN,NaN,NaN
20013,"""status"" : 500",NaT,NaN,NaN,NaN,NaN


In [396]:
# b_df = dask_df.drop(dask_df[dask_df.Borough != 'BROOKLYN'].index)
b_df = res[res.Borough == 'BROOKLYN']

In [397]:
b_df.rename(columns={"Incident Zip": "Zip"}, inplace = True)

/Users/epavlov/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/core/frame.py:5042: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [236]:
# list(zip_df['Zip'].unique())
zip_df.dtypes

Neighborhood    object
Zip              int64
dtype: object

In [398]:
b_df.dropna(subset=['Zip'], inplace=True)

/Users/epavlov/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [399]:
b_df = b_df[b_df['Zip'] != 'UNKNOWN']

In [400]:
b_df['Zip'] = b_df['Zip'].astype(np.int64)

In [401]:
b_df.shape

(3697317, 6)

In [241]:
b_df.dtypes

Unique Key                object
Created Date      datetime64[ns]
Complaint Type            object
Incident Zip               int64
City                      object
Borough                   object
dtype: object

In [402]:
b_df = b_df[b_df['Zip'].isin(list(zip_df['Zip'].unique()))]

In [403]:
b_df['Zip'].unique()

array([11236, 11214, 11249, 11210, 11203, 11237, 11212, 11230, 11219,
       11213, 11238, 11234, 11221, 11233, 11229, 11235, 11206, 11211,
       11201, 11207, 11217, 11223, 11220, 11232, 11215, 11226, 11209,
       11208, 11205, 11204, 11228, 11216, 11218, 11231, 11224, 11225,
       11222, 11239])

In [270]:
# df1 = dask_df.groupby(['Complaint Type']).filter(lambda x : len(x)>5000)
pop_df.dtypes

Zip           object
Population    object
dtype: object

In [288]:
b_df['Zip'].unique()

array([11236, 11214, 11210, 11203, 11237, 11212, 11230, 11219, 11213,
       11238, 11234, 11221, 11233, 11229, 11235, 11206, 11211, 11201,
       11207, 11217, 11223, 11220, 11232, 11215, 11226, 11209, 11208,
       11205, 11204, 11228, 11216, 11218, 11231, 11224, 11225, 11222,
       11239])

In [404]:
b_df[b_df["Complaint Type"].str.contains('Noise')]

,Unique Key,Created Date,Complaint Type,Zip,City,Borough
15,25804844,2013-06-25 01:00:13,Noise - Residential,11203,BROOKLYN,BROOKLYN
17,25804846,2013-06-24 21:01:18,Noise - Commercial,11237,BROOKLYN,BROOKLYN
52,25805047,2013-06-25 00:08:29,Noise - Commercial,11236,BROOKLYN,BROOKLYN
53,25805048,2013-06-24 16:15:02,Noise - Commercial,11219,BROOKLYN,BROOKLYN
54,25805049,2013-06-24 20:17:08,Noise - Residential,11213,BROOKLYN,BROOKLYN
...,...,...,...,...,...,...
19624,42609018,2019-05-07 14:46:00,Noise,11206,BROOKLYN,BROOKLYN
19791,42609267,2019-05-08 00:05:50,Noise - Residential,11237,BROOKLYN,BROOKLYN
19934,42609462,2019-05-07 21:46:10,Noise - Commercial,11213,BROOKLYN,BROOKLYN
19942,42609480,2019-05-07 11:47:37,Noise - Street/Sidewalk,11213,BROOKLYN,BROOKLYN


In [291]:
# dask_df.shape

In [405]:
b_df.groupby(['Complaint Type']).count()

,Unique Key,Created Date,Zip,City,Borough
Complaint Type,,,,,
AGENCY,1,1,1,1,1
APPLIANCE,19098,19098,19098,19098,19098
ATF,22,22,22,22,22
Adopt-A-Basket,104,104,104,104,104
Advocate - Levy,1,1,1,1,1
...,...,...,...,...,...
Water Conservation,6723,6723,6723,6723,6723
Water Quality,2302,2302,2302,2302,2302
Water System,106293,106293,106293,106293,106293


In [178]:
# df1.groupby(['Complaint Type']).count()

In [407]:
# df1['Complaint Type'].unique()

In [325]:
complaints = ['Illegal Parking', 
        'Noise - Commercial', 'Noise - Residential',
       'Derelict Vehicle',
       'Indoor Air Quality', 'Noise - Street/Sidewalk', 'Noise - Park',
       'Non-Emergency Police Matter', 'Noise - Vehicle',
       'Blocked Driveway', 'Drinking', 'Vending', 'Food Establishment',
       'BEST/Site Safety',
       'General Construction/Plumbing', 'Unsanitary Animal Pvt Property',
       'Sidewalk Condition', 'Air Quality', 'Noise', 
       'Lead', 'Street Condition',
       'Hazardous Materials','Curb Condition',
       'Root/Sewer/Sidewalk Condition',
       'Special Projects Inspection Team (SPIT)', 'Building/Use',
       'Graffiti', 'Construction', 
       'Bus Stop Shelter Complaint', 'Asbestos','Water Quality',
       'Industrial Waste', 'Emergency Response Team (ERT)',
       'Noise - House of Worship',
       'Homeless Encampment', 'Miscellaneous Categories', 
       'Derelict Vehicles', 'Sanitation Condition', 'Smoking',
       'Overflowing Litter Baskets', 'Homeless Person Assistance',
       'Other Enforcement', 'PAINT/PLASTER', 
       'Derelict Bicycle', 'GENERAL', 'Dirty Conditions',
       'ELECTRIC', 
       'Missed Collection (All Materials)', 'Litter Basket / Request',
       'Sweeping/Missed', 'SAFETY', 'UNSANITARY CONDITION', 
       'Broken Parking Meter',
       'Construction Safety Enforcement', 
       'Drug Activity',
       'DOF Property - Payment Issue', 'Building Marshals office',
       'Borough Office']

In [179]:
b_df.dtypes

Unique Key                object
Created Date      datetime64[ns]
Complaint Type            object
Incident Zip              object
City                      object
Borough                   object
dtype: object

In [180]:
b_df.head()


,Unique Key,Created Date,Complaint Type,Incident Zip,City,Borough
2,25804769,2013-06-24 10:16:53,Illegal Parking,11236,BROOKLYN,BROOKLYN
4,25804774,2013-06-24 09:26:03,Overgrown Tree/Branches,11214,BROOKLYN,BROOKLYN
5,25804775,2013-06-24 14:52:05,Illegal Tree Damage,11249,BROOKLYN,BROOKLYN
9,25804825,2013-06-24 01:10:00,Traffic Signal Condition,NaN,NaN,BROOKLYN
10,25804826,2013-06-24 12:29:00,Traffic Signal Condition,NaN,NaN,BROOKLYN


In [406]:
# df1.sort_values('Created Date', inplace=True)

In [408]:
b_df_copy = b_df.copy()

In [132]:
# df1 = df_copy.copy()

In [144]:
# df1.index.name = 'id'

In [121]:
# df1.set_index('Created Date', inplace=True)

In [292]:
# df1.reset_index()

In [293]:
# df1

In [409]:
# count_df = df1[['Creation Date','Complaint Type','Incident Zip']].groupby([pd.Grouper(key='Created Date', freq='Y'),'Complaint Type','Incident Zip'])['id'].count().reset_index(name="count")
count_b_df = b_df.groupby(by = ['Complaint Type', 'Zip', pd.Grouper(key='Created Date', freq='M')]).count()





In [410]:
count_b_df

Unique Key  City  Borough
Complaint Type          Zip   Created Date                           
AGENCY                  11204 2016-02-29             1     1        1
APPLIANCE               11201 2014-04-30             1     1        1
                              2014-05-31             1     1        1
                              2014-07-31             2     2        2
                              2014-09-30             5     5        5
...                                                ...   ...      ...
X-Ray Machine/Equipment 11220 2016-10-31             1     1        1
                        11229 2014-07-31             1     1        1
                        11230 2013-08-31             1     1        1
                        11235 2017-09-30             1     1        1
                        11249 2018-01-31             1     1        1

[214901 rows x 3 columns]

In [411]:
count_b_df.to_csv("b_counted_complaints.csv")

In [412]:
# resetting index did not help as after groupby the columns that were used for sorting stopped being columns
count_b_df = pd.read_csv("b_counted_complaints.csv")

In [413]:
# combine Brooklyn data set with population per zip code
count_b_df = pd.merge(count_b_df, pop_df, on="Zip")

In [359]:
count_b_df.shape

(214156, 7)

In [414]:
count_b_df = pd.merge(count_b_df, zip_df, on="Zip")
count_b_df.shape

(210025, 8)

In [415]:
count_b_df.columns

Index(['Complaint Type', 'Zip', 'Created Date', 'Unique Key', 'City',
       'Borough', 'Population', 'Neighborhood'],
      dtype='object')

In [416]:

# count_b_df = count_b_df.drop(['City', 'Borough'])
count_b_df.rename(columns={"Unique Key": "Count"}, inplace = True)


In [417]:
count_b_df['Incident Density'] = count_b_df['Count']*1000/count_b_df['Population']

In [378]:
# count_b_df['Complaint Type'].unique()

In [432]:
# for name in complaints: # count_b_df['Complaint Type'].unique():
name = "Noise"
px.line(count_b_df[count_b_df['Complaint Type']==name], x='Created Date', y='Incident Density', color='Neighborhood',title=name, 
#         labels={
#                  "Incident Density": name
#              }
       ).show()

In [567]:
noise_df = count_b_df[count_b_df['Complaint Type'].str.contains("Noise")]   
# count_b_df[count_b_df['Complaint Type'].str.contains("Noise")].groupby(by = ['Created Date', 'Zip']).sum()
noise_df = noise_df.drop(['City', 'Borough', 'Population', 'Incident Density', 'Neighborhood'], axis=1)
noise_df
                                                                             
                                                                             

,Complaint Type,Zip,Created Date,Count
643,Collection Truck Noise,11204,2016-02-29,1
644,Collection Truck Noise,11204,2016-11-30,2
645,Collection Truck Noise,11204,2016-12-31,1
646,Collection Truck Noise,11204,2017-10-31,1
647,Collection Truck Noise,11204,2018-06-30,1
...,...,...,...,...
209213,Noise - Vehicle,11239,2018-10-31,3
209214,Noise - Vehicle,11239,2019-01-31,1
209215,Noise - Vehicle,11239,2019-02-28,2
209216,Noise - Vehicle,11239,2019-03-31,1


In [568]:
all_noise_df = noise_df.groupby(by = ['Created Date', 'Zip']).sum()
all_noise_df.to_csv("noise.csv")
all_noise_df = pd.read_csv("noise.csv")

In [569]:
# before taking top 5 normalize by population. And then take top 5 in population density. 
total_noise_by_zip_df = noise_df.groupby(["Zip"]).sum()
total_noise_by_zip_df = pd.merge(total_noise_by_zip_df, pop_df, on="Zip")
total_noise_by_zip_df["Density per 100000 people"] = round(total_noise_by_zip_df["Count"]*100000/total_noise_by_zip_df["Population"],2)
top5zipcodes = total_noise_by_zip_df.groupby(["Zip"]).sum()["Density per 100000 people"].nlargest(n=5).index.tolist()
bottom5zipcodes = total_noise_by_zip_df.groupby(["Zip"]).sum()["Density per 100000 people"].nsmallest(n=5).index.tolist()


In [471]:
# noise_df.columns
# top5zipcodes = noise_df.groupby(["Zip"]).sum()['Count'].nlargest(n=5).index.tolist()

In [473]:
# noise_df = noise_df[noise_df['Zip'].isin(top5zipcodes)]

In [525]:
# noise_df

In [524]:
# noise_df['Zip'].unique()

In [570]:
all_noise_df = noise_df.groupby(by = ['Created Date', 'Zip']).sum()
all_noise_df.to_csv("all_noise.csv")
all_noise_df = pd.read_csv("all_noise.csv")

In [571]:

all_noise_df = pd.merge(all_noise_df, zip_df, on="Zip")


In [572]:
all_noise_df = pd.merge(all_noise_df, pop_df, on="Zip")


In [573]:
all_noise_df["Density per 100000 people"] = round(all_noise_df["Count"]*100000/all_noise_df["Population"],2)
all_noise_df = all_noise_df[all_noise_df['Zip'].isin(top5zipcodes+bottom5zipcodes)]
all_noise_df

,Created Date,Zip,Count,Neighborhood,Population,Density per 100000 people
144,2013-06-30,11204,21,Parkville/Bensonhurst,76395,27.49
145,2013-07-31,11204,112,Parkville/Bensonhurst,76395,146.61
146,2013-08-31,11204,109,Parkville/Bensonhurst,76395,142.68
147,2013-09-30,11204,99,Parkville/Bensonhurst,76395,129.59
148,2013-10-31,11204,85,Parkville/Bensonhurst,76395,111.26
...,...,...,...,...,...,...
2658,2019-01-31,11239,10,Starrett City,12772,78.30
2659,2019-02-28,11239,15,Starrett City,12772,117.44
2660,2019-03-31,11239,12,Starrett City,12772,93.96
2661,2019-04-30,11239,45,Starrett City,12772,352.33


In [574]:
px.line(all_noise_df, x='Created Date', y='Density per 100000 people', color='Neighborhood',title="All Noise").show()

In [637]:
rent_df = pd.read_csv("rent_cleaned.csv")
rent_df = rent_df.drop(["Borough", "areaType", "Slope"], axis=1)
rent_df = rent_df.drop(["Unnamed: 0"], axis=1)
rent_df = rent_df.set_index("Neighborhood")
rent_df = rent_df.T.stack(dropna=False)
rent_df.to_csv("test.csv")
rent_df = pd.read_csv("test.csv")
rent_df.columns = ["Created Date", "Neighborhood", "Price"]
rent_df.sort_values(by=["Neighborhood", "Created Date"])

,Created Date,Neighborhood,Price
0,2010-01,Bath Beach,NaN
61,2010-02,Bath Beach,NaN
122,2010-03,Bath Beach,NaN
183,2010-04,Bath Beach,NaN
244,2010-05,Bath Beach,NaN
...,...,...,...
8285,2021-04,Windsor Terrace,2500.0
8346,2021-05,Windsor Terrace,2200.0
8407,2021-06,Windsor Terrace,2400.0
8468,2021-07,Windsor Terrace,2648.0


In [646]:
rent_df['Created Date'] = pd.to_datetime(rent_df['Created Date'])
# to corrspond to the 311 data frame
rent_df['Created Date'] += pd.offsets.MonthEnd()
rent_df.dtypes
all_noise_df['Created Date'] = pd.to_datetime(all_noise_df['Created Date'])

In [640]:
px.line(rent_df, x='Created Date', y='Price', color='Neighborhood',title="All Rents").show()

In [651]:
all_noise_price_df = pd.merge(all_noise_df, rent_df, how='left', on=["Neighborhood", "Created Date"])
all_noise_price_df

,Created Date,Zip,Count,Neighborhood,Population,Density per 100000 people,Price
0,2013-06-30,11204,21,Parkville/Bensonhurst,76395,27.49,1500.0
1,2013-06-30,11204,21,Parkville/Bensonhurst,76395,27.49,2000.0
2,2013-07-31,11204,112,Parkville/Bensonhurst,76395,146.61,1425.0
3,2013-07-31,11204,112,Parkville/Bensonhurst,76395,146.61,1999.0
4,2013-08-31,11204,109,Parkville/Bensonhurst,76395,142.68,1238.0
...,...,...,...,...,...,...,...
786,2019-01-31,11239,10,Starrett City,12772,78.30,NaN
787,2019-02-28,11239,15,Starrett City,12772,117.44,NaN
788,2019-03-31,11239,12,Starrett City,12772,93.96,NaN
789,2019-04-30,11239,45,Starrett City,12772,352.33,NaN


In [652]:
all_noise_price_df[all_noise_price_df["Price"].isna()]

,Created Date,Zip,Count,Neighborhood,Population,Density per 100000 people,Price
432,2013-06-30,11225,46,Crown Heights,56859,80.90,NaN
433,2013-07-31,11225,284,Crown Heights,56859,499.48,NaN
434,2013-08-31,11225,347,Crown Heights,56859,610.28,NaN
435,2013-09-30,11225,332,Crown Heights,56859,583.90,NaN
436,2013-10-31,11225,301,Crown Heights,56859,529.38,NaN
...,...,...,...,...,...,...,...
786,2019-01-31,11239,10,Starrett City,12772,78.30,NaN
787,2019-02-28,11239,15,Starrett City,12772,117.44,NaN
788,2019-03-31,11239,12,Starrett City,12772,93.96,NaN
789,2019-04-30,11239,45,Starrett City,12772,352.33,NaN


In [660]:
all_noise_price_df.groupby('Zip')[['Density per 100000 people','Price']].corr()

Density per 100000 people     Price
Zip                                                                 
11204 Density per 100000 people                   1.000000  0.026103
      Price                                       0.026103  1.000000
11205 Density per 100000 people                   1.000000  0.014276
      Price                                       0.014276  1.000000
11214 Density per 100000 people                   1.000000  0.532158
      Price                                       0.532158  1.000000
11216 Density per 100000 people                   1.000000  0.036438
      Price                                       0.036438  1.000000
11219 Density per 100000 people                   1.000000  0.334374
      Price                                       0.334374  1.000000
11225 Density per 100000 people                   1.000000       NaN
      Price                                            NaN       NaN
11228 Density per 100000 people                   1.000000  0.393022
      Price                                       0.393022  1.000000
11237 Density per 100000 people                   1.000000  0.369787
      Price                                       0.369787  1.000000
11238 Density per 100000 people                   1.000000  0.026283
      Price                                       0.026283  1.000000
11239 Density per 100000 people                   1.000000       NaN
      Price                                            NaN       NaN

In [656]:
all_noise_price_df["Zip"].unique()

array([11204, 11205, 11214, 11216, 11219, 11225, 11228, 11237, 11238,
       11239])

In [657]:
all_noise_df["Zip"].unique()

array([11204, 11205, 11214, 11216, 11219, 11225, 11228, 11237, 11238,
       11239])

In [659]:
rent_df["Neighborhood"].unique()

array(['Bath Beach', 'Bay Ridge', 'Bedford-Stuyvesant',
       'Parkville/Bensonhurst', 'Bergen Beach', 'Boerum Hill',
       'Borough Park', 'Brighton Beach', 'Brooklyn', 'Brooklyn Heights',
       'Brownsville', 'Bushwick', 'Canarsie', 'Carroll Gardens',
       'Clinton Hill', 'Cobble Hill', 'Columbia St Waterfront District',
       'Coney Island', 'DUMBO', 'Ditmas Park', 'Downtown Brooklyn',
       'Dyker Heights', 'East Brooklyn', 'East Flatbush', 'East New York',
       'Vanderveer/Flatbush', 'Flatlands', 'Fort Greene', 'Gowanus',
       'Gravesend/Homecrest', 'Greenpoint', 'Greenwood', 'Kensington',
       'Manhattan Beach', 'Marine Park', 'Midwood', 'Mill Basin',
       'North Brooklyn', 'Northwest Brooklyn', 'Park Slope',
       'Prospect Heights', 'Prospect Lefferts Gardens', 'Prospect Park',
       'Prospect Park South', 'Red Hook', 'Sheepshead Bay',
       'South Brooklyn', 'Industry City/Sunset Park', 'Williamsburg',
       'Windsor Terrace', 'Bath Beach/Bensonhurst',
     